In [20]:
%matplotlib qt

from sklearnex import patch_sklearn
patch_sklearn()
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("../data/qbo_data.csv",index_col="run_id")
df = df.sort_values("qbo_period_std")
df

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


,cw,Bt,qbo_periods,qbo_period_std,qbo_period_fft,qbo_count,qbo_rmse_ampltiude,qbo_amplitude_mean,qbo_amplitude_std
run_id,,,,,,,,,
78,63.111004,0.002280,11.466667,3.984022e-15,11.466667,19,12.475006,17.731903,0.000000
1,44.617037,0.005099,24.116463,8.522248e-01,24.000000,9,42.536683,39.017867,1.996191
64,49.812074,0.005277,23.683767,8.823445e-01,24.000000,9,46.075949,42.475577,1.897165
60,21.845038,0.006956,19.117979,1.263149e+00,18.461538,12,34.874135,35.291554,1.546429
15,33.698484,0.005147,24.227832,1.284637e+00,24.000000,9,40.913906,38.525333,2.027852
...,...,...,...,...,...,...,...,...,...
35,8.580488,0.005694,18.281435,1.201578e+01,26.666667,12,12.293360,11.247630,4.733746
52,17.041039,0.001164,28.288639,1.384765e+01,12.000000,8,10.699096,11.159743,3.234233
44,5.930840,0.004727,27.009087,1.428539e+01,12.000000,8,10.644768,11.106186,1.924817


## Constants

In [21]:
OBS_PERIOD_MEAN = 27.915249015787193
OBS_PERIOD_STD = 4.538404428603899

OBS_AMP_MEAN = 26.587500000000002
OBS_AMP_STD =2.953799762291663

CURRENT_CW = 35
CURRENT_BT = 0.0043

In [22]:
import numpy as np
X = df[["Bt","cw"]].to_numpy()
y_point_error = df[["qbo_period_std","qbo_amplitude_std"]].to_numpy()
y = df[["qbo_periods","qbo_amplitude_mean"]].to_numpy()

mean_y = np.mean(y,axis=0)
std_y = np.std(y,axis=0)
y_norm = (y - mean_y)/std_y
y_point_error = y_point_error/std_y

In [23]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)




In [24]:
from sklearn.gaussian_process import GaussianProcessRegressor

gpr_period = GaussianProcessRegressor(alpha=(y_point_error[:,0])**2)
gpr_amplitude = GaussianProcessRegressor(alpha=(y_point_error[:,1])**2)
gpr_period.fit(X_scaled,y_norm[:,0])
gpr_amplitude.fit(X_scaled,y_norm[:,1])


GaussianProcessRegressor(alpha=array([0.        , 0.03549996, 0.03206519, 0.02130511, 0.03663503,
       0.10000635, 0.04755545, 0.02341443, 0.06873481, 0.02064695,
       0.04364164, 0.02059658, 0.03307179, 0.02997074, 0.02992444,
       0.03964912, 0.04184283, 0.03742722, 0.11043406, 0.05490023,
       0.03823451, 0.11996331, 0.01288341, 0.07016622, 0.01811312,
       0.02961625, 0.03895552, 0.03613724, 0.02275002, 0...
       0.06821497, 0.04865237, 0.03485085, 0.06325704, 0.40887736,
       0.18426707, 0.07807244, 0.05403737, 0.50658962, 0.11072214,
       0.25057328, 0.00183975, 0.24559099, 0.11531212, 0.0554244 ,
       0.58435664, 0.46871923, 0.11444193, 0.37895374, 0.05736947,
       0.45566798, 0.22893497, 0.07042133, 0.04721502, 0.12249928,
       0.07578465, 0.03409722, 0.0418027 , 0.04109161, 0.60180053,
       0.19963368, 0.09318942, 0.03300674, 0.06687922, 0.13468677]))

In [25]:
import numpy as np
cw = np.linspace(5,70,1000)
Bt = np.ones(len(cw))*0.0043
x_samples = np.array([Bt,cw]).T
x_samples = scaler.transform(x_samples)



In [26]:
samples,samples_std = gpr_period.predict(x_samples,return_std=True)
samples = samples*std_y[0] + mean_y[0]

In [27]:
# import matplotlib.pyplot as plt
# fig,(ax1,ax2)= plt.subplots(nrows=1,ncols=2,figsize=(10,6))

# ax1.plot(cw,samples)
# ax1.fill_between(
#     cw,
#     samples - 1.96*samples_std*std_y[0],
#     samples + 1.96*samples_std*std_y[0],
#     color="tab:blue",
#     alpha=0.5,
#     label=r"95% confidence interval",
# )
# ax1.axhline(OBS_MEAN,color='black',label="True Observation")
# ax1.fill_between(cw,OBS_MEAN-OBS_STD,OBS_MEAN+OBS_STD,color='black',alpha=0.3)
# ax1.set_xlabel("Cw tropics")
# ax1.set_ylabel("QBO Period (months)")
# ax1.errorbar(X[:,1],y[:,0],yerr=y_point_error[:,0]*std_y[0],fmt='o',label="MiMA runs")
# ax1.set_ylim(10,40)
# ax1.legend()

# ax2.plot(cw,implausability)
# ax2.axhline(np.percentile(implausability,20),c='g',label=f"20th percentile implausability cut off ")
# ax2.legend()
# ax2.set_ylabel("1D Implausability")
# ax2.set_xlabel("cw")

### 3D Plots?

In [28]:
CW_POINTS = 1000
BT_POINTS = 1000
cw = np.linspace(5,70,CW_POINTS)
Bt = np.linspace(0.001,0.007,BT_POINTS)
X_Bt,Y_cw = np.meshgrid(Bt,cw)
x_samples = np.vstack((X_Bt.ravel(),Y_cw.ravel())).T
x_samples = scaler.transform(x_samples)


In [29]:
period,period_std = gpr_period.predict(x_samples,return_std=True)
period = period*std_y[0] + mean_y[0]
period = period.reshape((CW_POINTS,BT_POINTS))
period_std = (period_std*std_y[0]).reshape((CW_POINTS,BT_POINTS))


amplitude,amplitude_std = gpr_amplitude.predict(x_samples,return_std=True)
amplitude = amplitude*std_y[1] + mean_y[1]
amplitude = amplitude.reshape((CW_POINTS,BT_POINTS))
amplitude_std = (amplitude_std*std_y[1]).reshape((CW_POINTS,BT_POINTS))

In [30]:
implausability_period = np.abs(period - OBS_PERIOD_MEAN)/np.sqrt(OBS_PERIOD_STD**2 + period_std**2)
implausability_amplitude = np.abs(amplitude - OBS_AMP_MEAN)/np.sqrt(OBS_AMP_STD**2 + amplitude_std**2)
implausability = np.sqrt(implausability_amplitude**2 + implausability_period**2)

In [31]:
imp_cutoff = implausability < np.percentile(implausability,20)

In [32]:
fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,6))
CS = ax1.contour(X_Bt,Y_cw,amplitude,levels=25)
ax1.scatter(X[:,0],X[:,1],)
ax1.scatter(CURRENT_BT,CURRENT_CW)
ax1.clabel(CS, CS.levels, inline=True,  fontsize=10)
ax1.set_ylabel("Cw")
ax1.set_xlabel("Bt")

CS2 = ax2.contour(X_Bt,Y_cw,period,levels=25)
ax2.scatter(X[:,0],X[:,1],)
ax2.scatter(CURRENT_BT,CURRENT_CW)
ax2.clabel(CS2, CS2.levels, inline=True,  fontsize=10)
ax2.set_ylabel("Cw")
ax2.set_xlabel("Bt")

ax3.contour(X_Bt,Y_cw,implausability,levels=25)
ax3.contourf(X_Bt,Y_cw,imp_cutoff,alpha=0.3,cmap='RdYlGn',levels=1)
ax3.scatter(X[:,0],X[:,1])
ax3.scatter(CURRENT_BT,CURRENT_CW)
ax3.set_ylabel("Cw")
ax3.set_xlabel("Bt")

Text(0.5, 0, 'Bt')

In [33]:
from mpl_toolkits.mplot3d import axes3d
ax = plt.figure(figsize=(10,10)).add_subplot(projection='3d')
ax.plot_surface(X_Bt,Y_cw,implausability,cmap='viridis')
ax.set_xlabel("Bt")
ax.set_ylabel("Cw")
ax.set_zlabel("Implausability")

Text(0.5, 0, 'Implausability')

## Calculate Decision Boundary + Sampling

In [34]:
def get_new_samples(new_sample_space,X,Y,n_samples=100):
    """
    MCMC sampling.
    Probably not very
    Dimensions of new_sample_space,X,Y, must match
    """
    ## Calculating bounding box of new_sample_space.
    y_where,x_where = np.where(new_sample_space)
    x_min,x_max,y_min,y_max = X_Bt[0,np.min(x_where)],X_Bt[0,np.max(x_where)],Y_cw[np.min(y_where),0],Y_cw[np.max(y_where),0]
    new_samples = []
    while len(new_samples) < n_samples:
        # Sample random point in original sample space 
        x,y = np.random.uniform(low=(x_min,y_min),high=(x_max,y_max))
        col = np.argmin(np.abs(X-x),axis=1)[1]
        row = np.argmin(np.abs(Y-y),axis=0)[0]
        if new_sample_space[row,col]:
            new_samples.append((x,y))
        # Check sample point meets plausability criterion
    return np.array(new_samples)


In [35]:
new_samples = get_new_samples(imp_cutoff,X_Bt,Y_cw)

In [36]:
fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,6))
CS = ax1.contour(X_Bt,Y_cw,amplitude,levels=25)
ax1.scatter(X[:,0],X[:,1],)
ax1.scatter(CURRENT_BT,CURRENT_CW)
ax1.clabel(CS, CS.levels, inline=True,  fontsize=10)
ax1.set_ylabel("Cw")
ax1.set_xlabel("Bt")

CS2 = ax2.contour(X_Bt,Y_cw,period,levels=25)
ax2.scatter(X[:,0],X[:,1],)
ax2.scatter(CURRENT_BT,CURRENT_CW)
ax2.clabel(CS2, CS2.levels, inline=True,  fontsize=10)
ax2.set_ylabel("Cw")
ax2.set_xlabel("Bt")

ax3.contour(X_Bt,Y_cw,implausability,levels=25)
ax3.contourf(X_Bt,Y_cw,imp_cutoff,alpha=0.3,cmap='RdYlGn',levels=1)
ax3.scatter(X[:,0],X[:,1])
ax3.scatter(CURRENT_BT,CURRENT_CW)
ax3.set_ylabel("Cw")
ax3.set_xlabel("Bt")
ax3.scatter(new_samples[:,0],new_samples[:,1],c='r')